# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time
import json
import googlemaps
import folium

# Import API key
from api_keys import geoapify_key
from api_keys import API_KEY


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
tortilla_df = pd.read_csv("Resources/new_tortilla.csv")
tortilla_df


,State,State.1,City,Year,Month,Day,Store type,Price per kilogram
0,113484,Aguascalientes,Aguascalientes,2014,1,3,Mom and Pop Store,12.00
1,113485,Baja California,Mexicali,2014,1,3,Mom and Pop Store,16.29
2,113486,Baja California,Tijuana,2014,1,3,Mom and Pop Store,13.55
3,113487,Baja California Sur,La Paz,2014,1,3,Mom and Pop Store,14.25
4,113488,Campeche,Campeche,2014,1,3,Mom and Pop Store,14.50
...,...,...,...,...,...,...,...,...
170594,289141,Veracruz,Coatzacoalcos,2024,10,21,Big Retail Store,12.57
170595,289142,Veracruz,Veracruz,2024,10,21,Big Retail Store,13.27
170596,289143,Veracruz,Xalapa,2024,10,21,Big Retail Store,13.73
170597,289144,Yucatán,Mérida,2024,10,21,Big Retail Store,13.13


In [4]:
#selectind the data just from 2024 since is the data of interest for the map
tortilla_2024 = tortilla_df[tortilla_df['Year'] >= 2024]
tortilla_2024

,State,State.1,City,Year,Month,Day,Store type,Price per kilogram
157641,276186,Aguascalientes,Aguascalientes,2024,1,3,Mom and Pop Store,21.67
157642,276187,Baja California,Mexicali,2024,1,3,Mom and Pop Store,30.57
157643,276188,Baja California,Tijuana,2024,1,3,Mom and Pop Store,25.64
157644,276189,Baja California Sur,La Paz,2024,1,3,Mom and Pop Store,27.00
157645,276190,Campeche,Campeche,2024,1,3,Mom and Pop Store,26.00
...,...,...,...,...,...,...,...,...
170594,289141,Veracruz,Coatzacoalcos,2024,10,21,Big Retail Store,12.57
170595,289142,Veracruz,Veracruz,2024,10,21,Big Retail Store,13.27
170596,289143,Veracruz,Xalapa,2024,10,21,Big Retail Store,13.73
170597,289144,Yucatán,Mérida,2024,10,21,Big Retail Store,13.13


In [5]:
#calculating how many cities are in the data
distinct_cities_count = tortilla_2024['City'].nunique()
distinct_cities_count

54

In [9]:
#getting an average price per city
average_price_city = tortilla_2024.groupby(['State.1', 'City'])['Price per kilogram'].mean().reset_index()
average_price_city.rename(columns={'Price per kilogram': 'Average Price per Kilogram'}, inplace=True)
average_price_city.head(10)

,State.1,City,Average Price per Kilogram
0,Aguascalientes,Aguascalientes,17.558333
1,Baja California,Mexicali,22.890458
2,Baja California,Tijuana,20.395083
3,Baja California Sur,La Paz,20.153917
4,Campeche,Campeche,19.118083
5,Chiapas,Tapachula,18.749125
6,Chiapas,Tuxtla Gutiérrez,17.483333
7,Chihuahua,Cd. Juárez,20.221417
8,Chihuahua,Chihuahua,22.650250
9,Coahuila,Piedras Negras,24.100417


In [10]:
#creating a new dataframe to get the lat and Long from the Geapify API
average_price_city["Lat"] = ""
average_price_city["Lon"] = ""
average_price_city.head(10)

,State.1,City,Average Price per Kilogram,Lat,Lon
0,Aguascalientes,Aguascalientes,17.558333,,
1,Baja California,Mexicali,22.890458,,
2,Baja California,Tijuana,20.395083,,
3,Baja California Sur,La Paz,20.153917,,
4,Campeche,Campeche,19.118083,,
5,Chiapas,Tapachula,18.749125,,
6,Chiapas,Tuxtla Gutiérrez,17.483333,,
7,Chihuahua,Cd. Juárez,20.221417,,
8,Chihuahua,Chihuahua,22.650250,,
9,Coahuila,Piedras Negras,24.100417,,


In [8]:
# Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [9]:
# Print a message to follow up the city search
print("Starting city search")

# Loop through the cities_pd DataFrame and search coordinates for each city
for index, row in average_price_city.iterrows():

    # Get the city's name & add ",Mexico" to the string so geoapify finds the correct city
    city = row["City"] + ", México"

    # Add the current city to the parameters
    params["text"] = f"{city}"

    # Make the API request
    response = requests.get(base_url, params=params)
    
    # Convert response to JSON
    response = response.json()

    # Extract latitude and longitude
    average_price_city.loc[index, "Lat"] = response["results"][0]["lat"]
    average_price_city.loc[index, "Lon"] = response["results"][0]["lon"]
    
    # Log the search results
    print(f"Coordinates for {city} fetched...")

# Display sample data to confirm that the coordinates appear
average_price_city.head()

Starting city search
Coordinates for Aguascalientes, México fetched...
Coordinates for Mexicali, México fetched...
Coordinates for Tijuana, México fetched...
Coordinates for La Paz, México fetched...
Coordinates for Campeche, México fetched...
Coordinates for Tapachula, México fetched...
Coordinates for Tuxtla Gutiérrez, México fetched...
Coordinates for Cd. Juárez, México fetched...
Coordinates for Chihuahua, México fetched...
Coordinates for Piedras Negras, México fetched...
Coordinates for Saltillo, México fetched...
Coordinates for Torreón, México fetched...
Coordinates for Colima, México fetched...
Coordinates for D.F., México fetched...
Coordinates for ZM D.F., México fetched...
Coordinates for Durango, México fetched...
Coordinates for Gómez Palacio, México fetched...
Coordinates for Toluca, México fetched...
Coordinates for Celaya, México fetched...
Coordinates for León, México fetched...
Coordinates for Acapulco, México fetched...
Coordinates for Chilpancingo, México fetched..

,State.1,City,Average Price per Kilogram,Lat,Lon
0,Aguascalientes,Aguascalientes,17.558333,21.880487,-102.296719
1,Baja California,Mexicali,22.890458,32.640525,-115.474899
2,Baja California,Tijuana,20.395083,32.53174,-117.019529
3,Baja California Sur,La Paz,20.153917,24.161995,-110.315853
4,Campeche,Campeche,19.118083,19,-90.5


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
# Configure the map plot
map_plot_1 = average_price_city.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    size = "Average Price per Kilogram",
    scale = 2.5,
    color = "City",
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Average Price per Kilogram)

In [11]:
sorted_price = average_price_city.sort_values(by='Average Price per Kilogram', ascending=False)
sorted_price_index = sorted_price.reset_index(drop=True)
sorted_price_index

,State.1,City,Average Price per Kilogram,Lat,Lon
0,Coahuila,Piedras Negras,24.100417,28.704596,-100.516714
1,Tamaulipas,Matamoros,23.221708,25.463614,-105.432334
2,Baja California,Mexicali,22.890458,32.640525,-115.474899
3,Sonora,Hermosillo,22.725750,29.094821,-110.96922
4,Chihuahua,Chihuahua,22.650250,25.543477,-99.955224
5,Sonora,San Luis Río Colorado,21.984708,32.451796,-114.765254
6,Tamaulipas,Reynosa,21.983250,26.090767,-98.278819
7,Guerrero,Acapulco,21.671292,17.962075,-94.761641
8,Sonora,Cd. Obregón,21.464292,27.484654,-109.935961
9,Tamaulipas,Nuevo Laredo,21.424750,15.563038,-92.385844


In [68]:
top_7_states = sorted_price_index.head(7)
top_7_states

,State.1,City,Average Price per Kilogram,Lat,Lon
0,Coahuila,Piedras Negras,24.100417,28.704596,-100.516714
1,Tamaulipas,Matamoros,23.221708,25.463614,-105.432334
2,Baja California,Mexicali,22.890458,32.640525,-115.474899
3,Sonora,Hermosillo,22.725750,29.094821,-110.96922
4,Chihuahua,Chihuahua,22.650250,25.543477,-99.955224
5,Sonora,San Luis Río Colorado,21.984708,32.451796,-114.765254
6,Tamaulipas,Reynosa,21.983250,26.090767,-98.278819


In [69]:
# Configure the map plot
map_plot_2 = top_7_states.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    size = "Average Price per Kilogram",
    scale = 2.5,
    color = "City",
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Average Price per Kilogram)

In [70]:
bottom_7_states = sorted_price_index.tail(7)

bottom_7_states

,State.1,City,Average Price per Kilogram,Lat,Lon
47,Chiapas,Tuxtla Gutiérrez,17.483333,16.753801,-93.115959
48,D.F.,D.F.,17.362500,19.43263,-99.133178
49,D.F.,ZM D.F.,17.011625,16.66806,-92.56861
50,Edo. México,Toluca,16.930000,19.292545,-99.656901
51,Puebla,Puebla,16.186125,16.91442,-92.503107
52,Tlaxcala,Tlaxcala,15.352417,19.416667,-98.166667
53,Puebla,ZM Puebla,15.145833,20.005045,-97.693422


In [71]:
# Configure the map plot
map_plot_3 = bottom_7_states.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    size = "Average Price per Kilogram",
    scale = 2.5,
    color = "City",
)

# Display the map plot
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Average Price per Kilogram)

In [16]:

# Define the base URL for the Google Places API
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

# Create a search query for tortillerías in Mexico
query = 'tortillerías in Mexico'

# Set up the parameters for the API request
params = {
    'query': query,
    'key': API_KEY
}

# Make the request to the Google Places API
response = requests.get(base_url, params=params)
response

<Response [200]>

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:


estados = {
    'Chihuahua': (28.6324, -106.0691),
    'Jalisco': (20.6597, -103.3496),
    'Nuevo León': (25.6714, -100.3080),
    'Yucatán': (20.9674, -89.5926),
    'Oaxaca': (17.0732, -96.7266),
    # Agrega otros estados con sus coordenadas
}

locations = []


# Importando Librería para verificar si está en México
def is_within_mexico(lat, lng):
    # Define los límites de México como ejemplos aproximados
    mexico_bounds = {
        'lat_min': 14.5320,  # Latitud mínima
        'lat_max': 32.7187,  # Latitud máxima
        'lng_min': -118.4043, # Longitud mínima
        'lng_max': -86.7106   # Longitud máxima
    }
    
    return (mexico_bounds['lat_min'] <= lat <= mexico_bounds['lat_max']) and (mexico_bounds['lng_min'] <= lng <= mexico_bounds['lng_max'])

for estado, coords in estados.items():
    query = f'tortillería {estado}'
    results = gmaps.places(query=query, location=coords, radius=20000, region="MX")
    for place in results.get('results', []):
        lat = place['geometry']['location']['lat']
        lng = place['geometry']['location']['lng']
        if is_within_mexico(lat, lng):  # Filtrar por límites de México
            locations.append({
                'name': place['name'],
                'lat': lat,
                'lng': lng,
                'estado': estado
            })

tortillerias_df_4 = pd.DataFrame(locations)

tortillerias_df_4

,name,lat,lng,estado
0,Tortilleria Chihuahua,28.625860,-106.062012,Chihuahua
1,Tortillería Santo Niño,28.649662,-106.077554,Chihuahua
2,Tortillería La 30,28.620298,-106.092725,Chihuahua
3,Tortillería Sta. Rosa,28.628574,-106.063529,Chihuahua
4,Tortillería Las Delicias,28.677051,-106.116326,Chihuahua
...,...,...,...,...
92,Tortillería La Soledad,17.003166,-96.758826,Oaxaca
93,Tortillas Lilia,17.102164,-96.748731,Oaxaca
94,Tortillería Elvis,17.080609,-96.706186,Oaxaca
95,Tortilleria La Soledad Xoxo,17.029270,-96.726442,Oaxaca


In [24]:
# Configure the map plot
map_plot_tort_4 = tortillerias_df_4.hvplot.points(
    "lng",
    "lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    scale = 5,
    color = "name",
)

# Display the map plot
map_plot_tort_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lng,lat]   (name)

In [25]:

map_center = [tortillerias_df_4['lat'].mean(), tortillerias_df_4['lng'].mean()]
map_tortillerias = folium.Map(location=map_center, zoom_start=12)

# Agregar puntos para cada tortillería
for _, row in tortillerias_df_4.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=row['name'],  # Nombre de la tortillería en el popup
        icon=folium.Icon(color='blue')  # Puedes cambiar el color según desees
    ).add_to(map_tortillerias)

# Mostrar el mapa
map_tortillerias

In [72]:
locations = []

# Importing Library to check if it is in Mexico
def is_within_mexico(lat, lng):
    # Define the approximate bounds of Mexico
    mexico_bounds = {
        'lat_min': 14.5320,  # Minimum latitude
        'lat_max': 32.7187,  # Maximum latitude
        'lng_min': -118.4043, # Minimum longitude
        'lng_max': -86.7106   # Maximum longitude
    }
    
    return (mexico_bounds['lat_min'] <= lat <= mexico_bounds['lat_max']) and (mexico_bounds['lng_min'] <= lng <= mexico_bounds['lng_max'])

# Looping through the rows of top_10_states to get tortillerías
for index, row in top_7_states.iterrows():
    estado = row['State.1']
    coords = (row['Lat'], row['Lon'])
    query = f'tortillería {estado}'
    results = gmaps.places(query=query, location=coords, radius=20000, region="MX")
    
    for place in results.get('results', []):
        lat = place['geometry']['location']['lat']
        lng = place['geometry']['location']['lng']
        if is_within_mexico(lat, lng):  # Filter by Mexico's limits
            locations.append({
                'name': place['name'],
                'lat': lat,
                'lng': lng,
                'estado': estado
            })

# Create a DataFrame for tortillerías
tortillerias_df_5 = pd.DataFrame(locations)

# Display the resulting DataFrame
tortillerias_df_5

,name,lat,lng,estado
0,Tortilleria COAHUILA,20.701395,-103.352564,Coahuila
1,Tortillería Alejandra,25.521601,-103.430551,Coahuila
2,Tortillería La Tapatia,25.548937,-103.435099,Coahuila
3,Maizito Tortillería,25.557959,-103.387671,Coahuila
4,Tortillería La Paloma,25.427446,-100.996524,Coahuila
...,...,...,...,...
66,Tortilleria tamaulipas #2,29.594051,-95.468218,Tamaulipas
67,Tortilleria Rosita,26.093910,-98.299033,Tamaulipas
68,Tortillería Aries,26.078152,-98.285366,Tamaulipas
69,Tortilleria Tamaulipas,24.853816,-99.550847,Tamaulipas


In [73]:
locations = []

# Function to check if a location is within Mexico's bounds
def is_within_mexico(lat, lng):
    mexico_bounds = {
        'lat_min': 14.5320,  # Minimum latitude
        'lat_max': 32.7187,  # Maximum latitude
        'lng_min': -118.4043, # Minimum longitude
        'lng_max': -86.7106   # Maximum longitude
    }
    
    return (mexico_bounds['lat_min'] <= lat <= mexico_bounds['lat_max']) and (mexico_bounds['lng_min'] <= lng <= mexico_bounds['lng_max'])

# Looping through the rows of top_10_states to fetch tortillerías
for index, row in top_7_states.iterrows():
    estado = row['State.1']
    coords = (row['Lat'], row['Lon'])
    query = f'tortillería {estado}'
    
    # Query the Google Places API for tortillerías in the area
    results = gmaps.places(query=query, location=coords, radius=20000, region="MX")
    
    for place in results.get('results', []):
        lat = place['geometry']['location']['lat']
        lng = place['geometry']['location']['lng']
        
        # Check if the location is within the bounds of Mexico
        if is_within_mexico(lat, lng):  # Filter by Mexico's limits
            locations.append({
                'name': place['name'],
                'lat': lat,
                'lng': lng,
                'estado': estado
            })
        else:
            print(f"Filtered out place: {place['name']} at ({lat}, {lng}) - Outside of Mexico")

# Create a DataFrame for tortillerías
tortillerias_df_6 = pd.DataFrame(locations)

# Display the resulting DataFrame
tortillerias_df_6

,name,lat,lng,estado
0,Tortilleria COAHUILA,20.701395,-103.352564,Coahuila
1,Tortillería Alejandra,25.521601,-103.430551,Coahuila
2,Tortillería La Tapatia,25.548937,-103.435099,Coahuila
3,Maizito Tortillería,25.557959,-103.387671,Coahuila
4,Tortillería La Paloma,25.427446,-100.996524,Coahuila
...,...,...,...,...
66,Tortilleria tamaulipas #2,29.594051,-95.468218,Tamaulipas
67,Tortilleria Rosita,26.093910,-98.299033,Tamaulipas
68,Tortillería Aries,26.078152,-98.285366,Tamaulipas
69,Tortilleria Tamaulipas,24.853816,-99.550847,Tamaulipas


In [76]:
tortillerias_df_dropped = tortillerias_df_6[tortillerias_df_6['name'] != 'Tortilleria Tamaulipas']
tortillerias_df_dropped

,name,lat,lng,estado
0,Tortilleria COAHUILA,20.701395,-103.352564,Coahuila
1,Tortillería Alejandra,25.521601,-103.430551,Coahuila
2,Tortillería La Tapatia,25.548937,-103.435099,Coahuila
3,Maizito Tortillería,25.557959,-103.387671,Coahuila
4,Tortillería La Paloma,25.427446,-100.996524,Coahuila
...,...,...,...,...
65,Tortillería La Tamaulipas,25.666803,-100.136046,Tamaulipas
66,Tortilleria tamaulipas #2,29.594051,-95.468218,Tamaulipas
67,Tortilleria Rosita,26.093910,-98.299033,Tamaulipas
68,Tortillería Aries,26.078152,-98.285366,Tamaulipas


In [78]:
# Configure the map plot
map_plot_tort_5 = tortillerias_df_dropped.hvplot.points(
    "lng",
    "lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    scale = 5,
    color = "name",
)

# Display the map plot
map_plot_tort_5

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lng,lat]   (name)

In [79]:

map_center = [tortillerias_df_dropped['lat'].mean(), tortillerias_df_dropped['lng'].mean()]
map_tortillerias_2 = folium.Map(location=map_center, zoom_start=5)

# Agregar puntos para cada tortillería
for _, row in tortillerias_df_dropped.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=row['name'],  # Nombre de la tortillería en el popup
        icon=folium.Icon(color='blue')  # Puedes cambiar el color según desees
    ).add_to(map_tortillerias_2)

# Mostrar el mapa
map_tortillerias_2

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

### Extra: Clean the ideal weather dataframe and drop the No Hotel Found rows to display the chart 
### only with cities in which hotels could be found.